In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import sys
print(sys.version)

3.7.1 (default, Dec 14 2018, 19:28:38) 
[GCC 7.3.0]


In [2]:
train=pd.read_csv('train.csv',sep='|')
test=pd.read_csv('test.csv',sep='|')

In [11]:
train.head()

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud
0,5,1054,54.70,7,0,3,0.027514,0.051898,0.241379,0
1,3,108,27.36,5,2,4,0.129630,0.253333,0.357143,0
2,3,1516,62.16,3,10,5,0.008575,0.041003,0.230769,0
3,6,1791,92.31,8,4,4,0.016192,0.051541,0.275862,0
4,5,430,81.53,3,7,2,0.062791,0.189605,0.111111,0


In [4]:
train.dtypes

trustLevel                     int64
totalScanTimeInSeconds         int64
grandTotal                   float64
lineItemVoids                  int64
scansWithoutRegistration       int64
quantityModifications          int64
scannedLineItemsPerSecond    float64
valuePerSecond               float64
lineItemVoidsPerPosition     float64
fraud                          int64
dtype: object

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
y_train= train['fraud']
x_train= train.drop('fraud',axis=1)


x_train, x_holdout, y_train, y_holdout = train_test_split(x_train, y_train, test_size=0.2, random_state=324)

model= DecisionTreeClassifier(max_leaf_nodes=10, random_state=0)
model.fit(x_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=10,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [6]:
from sklearn.metrics import accuracy_score

predictions=model.predict(x_holdout)
accuracy_score(y_true = y_holdout, y_pred = predictions)

0.949468085106383

In [7]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_holdout,predictions)
cm

array([[344,   7],
       [ 12,  13]])

In [8]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_holdout,predictions)

0.75002849002849

In [9]:
def get_monetary_value(cm, y_holdout):
        tn, fp, fn, tp = cm.ravel()
        print("True negative: ", tn)
        print("False positive: ", fp)
        print("False negative: ", fn)
        print("True positive: ", tp)
        score = (-25)*fn + (-5)*fp + 5*tp
        print(score, 'over',y_holdout[y_holdout > 0].count()*5)
        return score
        
        
monetary_value = get_monetary_value(cm, y_holdout)

True negative:  344
False positive:  7
False negative:  12
True positive:  13
-270 over 125
